In [1]:
# 내 버킷 리스트 확인
# gsutil -> google storage util
# understand_gcp_bucket참조
!gsutil list

gs://cloud-ai-platform-8e31d2d5-4bf8-43ba-980d-604814f702a7/
gs://highway_test/


In [6]:
BUCKET_GS_NAME="gs://highway_test/"

In [3]:
# 버킷안에 파일 목록 확인
#!gsutil ls -r {BUCKET_NAME}

gs://highway_test/.ipynb_checkpoints/:
gs://highway_test/.ipynb_checkpoints/

gs://highway_test/HighwayTest/:
gs://highway_test/HighwayTest/교통문제CCTV영상.z01
gs://highway_test/HighwayTest/교통문제CCTV영상.z02
gs://highway_test/HighwayTest/교통문제CCTV영상.z03
gs://highway_test/HighwayTest/교통문제CCTV영상.z04
gs://highway_test/HighwayTest/교통문제CCTV영상.z05
gs://highway_test/HighwayTest/교통문제CCTV영상.z06
gs://highway_test/HighwayTest/교통문제CCTV영상.z07
gs://highway_test/HighwayTest/교통문제CCTV영상.z08
gs://highway_test/HighwayTest/교통문제CCTV영상.z09
gs://highway_test/HighwayTest/교통문제CCTV영상.z10
gs://highway_test/HighwayTest/교통문제CCTV영상.z11
gs://highway_test/HighwayTest/교통문제CCTV영상.z12
gs://highway_test/HighwayTest/교통문제CCTV영상.z13
gs://highway_test/HighwayTest/교통문제CCTV영상.z14
gs://highway_test/HighwayTest/교통문제CCTV영상.z15
gs://highway_test/HighwayTest/교통문제CCTV영상.z16
gs://highway_test/HighwayTest/교통문제CCTV영상.z17
gs://highway_test/HighwayTest/교통문제CCTV영상.zip
gs://highway_test/HighwayTest/교통문제CCTV영상all.zip

gs://highway_test/HighwayTest/.

## 구글 드라이브 마운트 하듯이 버킷을 마운트 하는 방법

In [4]:
BUCKET_NAME='highway_test'
MOUNT_PATH ='/home/jupyter/test' 
!gcsfuse --implicit-dirs {BUCKET_NAME} {MOUNT_PATH}

{"time":"22/11/2023 01:29:07.275117","severity":"INFO","msg":"Start gcsfuse/1.2.0 (Go version go1.21.0) for app \"\" using mount point: /home/jupyter/test\n"}


In [12]:
!gsutil du -s {BUCKET_GS_NAME}

89742581185  gs://highway_test


In [8]:
import os
HOME=os.getcwd()
print(HOME)

/home/jupyter


In [8]:
# %cd {HOME}
# !git clone https://github.com/ultralytics/ultralytics
# %cd ultralytics
!pip install ultralytics

In [9]:
!ls -al

total 5868
drwxr-xr-x 18 jupyter jupyter    4096 Nov 22 01:29 .
drwxr-xr-x  3 root    root       4096 Nov 15 06:42 ..
-rw-------  1 jupyter jupyter     573 Nov 22 00:13 .bash_history
-rw-r--r--  1 jupyter jupyter     431 Nov 15 06:44 .bashrc
drwxr-xr-x  7 jupyter jupyter    4096 Nov 21 06:30 .cache
drwxr-xr-x  5 jupyter jupyter    4096 Nov 16 07:03 .config
drwxr-xr-x  2 jupyter jupyter    4096 Nov 15 06:44 .docker
drwxr-xr-x  2 jupyter jupyter    4096 Nov 21 04:55 .gsutil
drwxr-xr-x  2 jupyter jupyter    4096 Nov 21 08:48 .ipynb_checkpoints
drwxr-xr-x  3 jupyter jupyter    4096 Nov 15 06:44 .ipython
drwxr-xr-x  3 jupyter jupyter    4096 Nov 15 06:53 .jupyter
drwxr-xr-x  3 jupyter jupyter    4096 Nov 15 06:44 .local
drwxr-xr-x  3 jupyter jupyter    4096 Nov 15 06:44 .npm
drwx------  3 jupyter jupyter    4096 Nov 15 07:18 .nv
-rw-r--r--  1 jupyter jupyter    3759 Nov 21 06:52 Untitled1.ipynb
drwxr-xr-x  3 jupyter jupyter    4096 Nov 21 08:50 datasets
-rw-------  1 jupyter jupyter  106055

In [10]:
#!gsutil ls -L "gs://highway_test/HighwayTest" 

In [10]:
#%cd /home/jupyter/datasets/dataset/HighwayTest/
%cd /home/jupyter/test/HighwayTest/
!pwd

/home/jupyter/test/HighwayTest
/home/jupyter/test/HighwayTest


In [13]:
!ls

Train  Validation


## 분할압축파일 풀때  7zip이 젤 나아보임

In [ ]:
# !sudo apt install p7zip-full
# !7za x 교통문제CCTV영상.zip

In [ ]:
# train - images
#         labels

# valid - images
#         labels 

## 레이블 파일을 xml -> txt

In [18]:
!pip install lxml

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.9/7.9 MB 53.9 MB/s eta 0:00:0000:0100:01


In [26]:
import numpy as np
import lxml
import os

CLASSES = ["car", "bus", "truck"]

from lxml import etree

# ms coco데이터셋 타입의 레이블 -> YOLO모델 타입으로 레이블 변경
def to_yolov8(y):
  """
  # change to yolo v8 format
  # [x_top_left, y_top_left, x_bottom_right, y_bottom_right] to
  # [x_center, y_center, width, height]
  """
  width = y[2] - y[0]
  height = y[3] - y[1]

  if width < 0 or height < 0:
      print("ERROR: negative width or height ", width, height, y)
      raise AssertionError("Negative width or height")
  return (y[0] + (width/2)), (y[1] + (height/2)), width, height

def load_xml_annotations(f):
  tree = etree.parse(f)
  anns = []
  for dim in tree.xpath("image"):
    image_filename = dim.attrib["name"]
    width = int(dim.attrib["width"])
    height = int(dim.attrib["height"])
    # print(image_filename)
    # print(len(dim.xpath("box")))
    boxes = []
    for box in dim.xpath("box"):
      label = CLASSES.index(box.attrib["label"])
      xtl, ytl = box.attrib["xtl"], box.attrib["ytl"]
      xbr, ybr = box.attrib["xbr"], box.attrib["ybr"]

      xc, yc, w, h = to_yolov8([float(xtl), float(ytl), float(xbr), float(ybr)])
      boxes.append([label, round(xc/width, 5), round(yc/height, 5), round(w/width, 5), round(h/height, 5)])

    anns.append([image_filename, width, height, boxes])

  return np.array(anns, dtype='object')


def write_yolov8_txt(folder, annotation):
  #print(annotation[0][:-3])
  out_filename = os.path.join(folder,str(annotation[0][:-3]))
  out_filename = os.path.splitext(out_filename)[0]
  out_filename = out_filename+'.txt'

  f = open(out_filename,"w+")
  for box in annotation[3]:
    f.write("{} {} {} {} {}\n".format(box[0], box[1], box[2], box[3], box[4]))


## 한개 XML을  폴더를 만들고 TXT를 생성

In [28]:
import os

dataPath='/home/jupyter/test/HighwayTest/Validation/라벨V수도권영동선/'
dstPath='/home/jupyter/test/HighwayTest/Validation/labels/'
# 파일명을 설정한다.
xmlFile='Suwon_CH01_20200720_1830_MON_9m_RH_highway_TW5_sunny_FHD.xml'

# 현재 경로 + 파일명
label_file = os.path.join(dataPath, xmlFile)

# XML을 TXT로 변환한다.
anns = load_xml_annotations(label_file)
print(anns)

folderName=os.path.splitext(xmlFile)[0]
os.makedirs(os.path.join(dstPath,folderName), exist_ok=True)


# # 이미지 파일별로 TXT파일을 저장한다.
for ann in anns:
  write_yolov8_txt(dstPath+folderName, ann)

## 여러개 XML을 폴더를 만들고 TXT를 생성

In [32]:
from glob import glob
xmlList=glob(dataPath+'*.xml')
# xmlList
# for xml in xmlList:    
#     print(xml)

In [33]:
# 이미지파일과 레이블 파일을 images와 labels로 복사